
# Incorporating OLI Calculations with WaterTAP

#### Contact: Paul Vecchiarelli (paul.vecchiarelli@nrel.gov)

This tutorial will demonstrate basic usage of OLI Cloud calls using our custom API tools.

## Rationale

 - Simulations for realistic water sources are mathematically complex: 
 > $ Interactions \ge Cations * Anions$
 - OLI improves WaterTAP approximations and offloads computational resources

## Required OLI API Inputs


 - State variables (solute concentrations, temperature, pressure), which can be extracted from a state block
 
 - Login credentials
 
 - A chemistry (*.dbs) file
     - establishes state variables, phases, etc. to be considered in flash calls

In [ ]:
# used to build survey lists
from numpy import linspace

# used to execute OLI Cloud functions
from watertap.tools.oli_api.flash import (
    Flash,
    build_survey,
    get_survey_sample_conditions,
    write_output,
)
from watertap.tools.oli_api.credentials import CredentialManager
from watertap.tools.oli_api.client import OLIApi

# 1. Specify State Variables.

- This data is used to construct inputs to OLI Cloud
- Same basic information is required by Water Analysis and Isothermal flashes (temperature, pressure, solutes)

In [ ]:
inflows = {
    "Na_+": 1200,
    "Cl_-": 1800,
    "Ca_2+": 200,
    "SO4_2-": 300,
}

# 2. Initialize Flash Instance.

 - We will run most of our methods with this class

In [ ]:
flash = Flash(debug_level="INFO")

# 3. Get Survey Parameters.

 - In this example, we will generate a temperature sweep survey

In [ ]:
# a survey will sweep through one or more variables simultaneously
# surveys are enabled for concentration, pH, temperature, pressure, and other variables

# using the default grid_mesh setting,
# build_survey computes the Cartesian product of the input arrays
survey = build_survey(
    {
        "Na_+": linspace(0, 1e4, 2),
        "Cl_-": linspace(0, 1e4, 5),
    },
    get_oli_names=True,
    mesh_grid=True,
    file_name="oli_survey.json",
)

# uncomment the following lines to view results:
#print("Survey using grid_mesh")
#print(survey)

# setting mesh_grid=False allows custom surveys to be defined, i.e., no Cartesian product is computed.
survey_custom = build_survey(
    {
         "Na_+": [0., 0., 0., 0., 0., 1e4, 1e4, 1e4, 1e4, 1e4],
         "Cl_-": [0., 2.5e3, 5e3, 7.5e3, 1e4, 0., 2.5e3, 5e3, 7.5e3, 1e4],
    },
    get_oli_names=True,
    mesh_grid=False,
    file_name="oli_survey_custom.json",
)

# uncomment the following lines to view results:
#print("Survey with custom sampling")
#print(survey_custom)

In [ ]:
# individual sample points can be accessed to see what will be modified
samples = [0,1,2,3,4,5,6,7,8,9]
survey_points = get_survey_sample_conditions(survey, samples)
survey_points_custom = get_survey_sample_conditions(survey_custom, samples)

# uncomment the following lines to view results:
#for sample in samples:
#    print(f"sample: {sample}")
#    print(f"grid survey point: {survey_points[sample]}")
#    print(f"custom survey point: {survey_points_custom[sample]}")

write_output(survey_points, "test_survey_points.json")
write_output(survey_points_custom, "test_survey_points_custom.json")

# 4. Login to OLI Cloud.

- The following code demonstrates an OLI Cloud login.

There are 3 ways to login:
1. username/password
2. access key
3. load credentials from file

On your first login, use method 1 (below).
You can then generate access keys and save credentials.

In [ ]:
import os 

# there are 3 intended methods of logging in:
# 1: using username, password, root_url, and auth_url
# 2: using access_keys and root_url
# 3: using encryption_key to load credentials from config_file

# methods 1 and 2 will prompt to save credentials and provide an encryption key to use with method 3.

# NOTE: `interactive_mode` in CredentialManager works like `debug_level` in OLIApi and Flash
# while `test` in CredentialManager is equivalent to `interactive_mode` in OLIApi
'''
# method 1: input username, password, root_url, and auth_url
credential_manager = CredentialManager(
    username="",
    password="",
    root_url="",
    auth_url="",
    interactive_mode=True,
    test=True,
)
'''
# when prompted, allow WaterTAP to save credentials to generate encryption key.

# to overwrite saved credentials, create a new credential_manager instance with 
# the credentials to write and permit credentials to be save to the config_file.
# e.g., new_credential_manager = CredentialManager(credential_manager.credentials)

# method 2: input access_keys and root_url

# credential_manager.generate_oliapi_access_key will create a new access key and add it to
# credential_manager.credentials["access_keys"].

# credential_manager.delete_oliapi_access_key will delete a specified key.

# NOTE: by default, this method is enabled in the tutorial for testing purposes.
# Change test to False to enable prompts with CredentialManager.
credential_manager = CredentialManager(
    access_keys=[os.environ["OLI_API_KEY"]],
    root_url=os.environ["OLI_API_ROOT_URL"],
    interactive_mode=True,
    test=True,
)
'''
# method 3: input encryption_key
credential_manager = CredentialManager(
    encryption_key="",
    interactive_mode=True,
    test=True,
)
'''
# NOTE: a new encryption key will be generated every time the config_file is overwritten 

# 5. Process Flash Calculations.

In [ ]:
# OLIApi is used as a context manager
with OLIApi(credential_manager, interactive_mode=False, debug_level="INFO") as oliapi:
    # create a new DBS file
    # - alternative thermo_frameworks and databanks are available.
    # - use keep_file = True to save a DBS file ID on the Cloud
    # for use in more than one session
    dbs_file_id = oliapi.generate_dbs_file(
        inflows=inflows,
        thermo_framework="MSE (H3O+ ion)",
        phases=["liquid1", "solid"],
        model_name="test",
        keep_file=True,
    )
    # NOTE: liquid2 (non-aqueous) and vapor phases are available but not yet fully supported.

In [ ]:
# Flash calculation example #1: Water Analysis survey with pH reconciliation.

# OLIApi is used as a context manager
with OLIApi(credential_manager, interactive_mode=False, debug_level="INFO") as oliapi:

    # create water analysis input
    # Water Analysis uses true species for inputs, i.e., IONS.
    json_input = flash.configure_water_analysis(
        inflows,
        reconciliation="ReconcilePh",
        ph=5,
        allow_solids=True,
        file_name="water_analysis_inputs.json",
    )
    # run Water Analysis flash calculation survey as specified        
    stream_output = flash.run_flash(
        "wateranalysis",
        oliapi,
        dbs_file_id,
        json_input,
        survey,
        file_name="water_analysis_outputs.json",
    )

    # The output of Water Analysis gives apparent species
    # i.e., SALTS, COMPLEXES (MgO, CaO, etc.)

In [ ]:
# Flash calculation example #2: isothermal analysis survey
with OLIApi(credential_manager, interactive_mode=False, debug_level="INFO") as oliapi:

    # Other flash calculations use apparent species for inputs.

    # These can be obtained in one step with get_apparent_species_from_true(),
    # and saved to a file and reloaded as needed.
    apparent_species = flash.get_apparent_species_from_true(
        json_input, oliapi, dbs_file_id, file_name="apparent_species.json"
    )
    isothermal_input = flash.configure_flash_analysis(
        apparent_species, "isothermal", file_name="isothermal_inputs.json"
    )
    # specify a new survey to compare apparent species
    isothermal_survey = build_survey(
        {"NaCl": linspace(0, 1e6, 10)},
        get_oli_names=True,
        file_name="isothermal_survey.json",
    )
    flash.run_flash(
        "isothermal",
        oliapi,
        dbs_file_id,
        isothermal_input,
        isothermal_survey,
        file_name="isothermal_outputs.json",
    )

In [ ]:
# Additional OLIApi functions to know:

with OLIApi(credential_manager, interactive_mode=False, debug_level="INFO") as oliapi:
    # all of a user's DBS files can be fetched from OLI Cloud
    dbs_files = oliapi.get_user_dbs_file_ids()
    
    # uncomment the following lines to view results:
    #for idx, file in enumerate(dbs_files):
    #    print(f"{idx+1}\t{file}")
    
    # if a DBS file has been retained from a previous session,
    # its flash history and chemistry information can be summarized.
    file_summary = oliapi.get_dbs_file_summary(dbs_file_id)

    # save chemistry information
    chemistry_info = file_summary["chemistry_info"]
    write_output(chemistry_info["result"], "chemistry_info.json")
    
    # DBS files can also be deleted.
    oliapi.dbs_file_cleanup(dbs_files)

# uncomment the following lines to view results:
#print(file_summary)

# 6. Extract Results.

In [ ]:
# There are 3 keys for results:
# 'submitted_requests' stores the JSON input for each Flash calculation
# 'metaData' stores metadata for each Flash calculation
# 'result' stores the JSON output for each Flash calculation

import json
from pandas import Series, DataFrame

with open("water_analysis_outputs.json", "rb") as json_output:
    results = json.load(json_output)["result"]

# uncomment the following lines to view results:
#print("Available keys:")
#for k in results:
#    print(f"- {k}")
#print()

In [ ]:
# example #1 extraction
prop = "selfDiffusivities_liquid1"
result = results[prop]
dataframe_input = []
units = ""
for k,v in result.items():
    if not units:
        units = v["units"]
    dataframe_input.append(Series(name=k, data=v["values"], dtype='Float64'))
    
# uncomment the following lines to view results:
#print(f"Result for {prop} ({units}):")
#DataFrame(dataframe_input)

In [ ]:
# example #1 extraction, part 2
prop = "enthalpy_total"
result = results[prop]
dataframe_input = []
units = result["units"]

# uncomment the following lines to view results:
#print(f"Result for {prop} ({units}):")
#Series(result["values"])

In [ ]:
# example #2 extraction
with open("isothermal_outputs.json", "rb") as json_output:
    results = json.load(json_output)["result"]

prop = "prescalingTendencies"
result = results[prop]
dataframe_input = []
units = ""
for k,v in result.items():
    if not units:
        units = v["units"]
    dataframe_input.append(Series(name=k, data=v["values"], dtype='Float64'))

# uncomment the following lines to view results:    
#print(f"Result for {prop} ({units}):")
#DataFrame(dataframe_input)